In [4]:
from tkinter import messagebox
from PIL import Image
import socket
import tkinter as tk
import numpy as np
import struct

window = tk.Tk()
window.geometry("700x700")
window.title("Login")
window.resizable(False,False)

# load the image file
try:
    bg_image = tk.PhotoImage(file='bgg.png')
except tk.TclError as e:
    print(f"Error loading image file: {e}")
else:
    # create a label with the PhotoImage as the background image
    bg_label = tk.Label(window, image=bg_image)
    bg_label.place(x=0, y=0, relwidth=1, relheight=1)

username_value = tk.StringVar()
password_value = tk.StringVar()

def clicked():
    messagebox.showwarning('Invalid', 'Incorrect Password!')

def submit_login():
    username=username_value.get()
    password=password_value.get()

    if(username=="admin" and password=="admin"):
        print("You are now logged in")
        window.destroy()

        # Create a datagram socket
        UDPServerSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)

        # Bind to address and ip
        UDPServerSocket.bind(("127.0.0.1", 20001))

        print("UDP server up and listening")

        connection,addr = UDPServerSocket.recvfrom(1024)
        print("\n"+connection.decode())

        image_ext,addr= UDPServerSocket.recvfrom(1024)
        print("\nImage Extension: "+image_ext.decode())

        # Receive file header,The length of the file header
        count=0
        packet_loss=0
        packet_lpercent=0
        packet_spercent=0

        # receive size and data
        recv_size=0
        data_size=0
        data_total=b""

        while True : 
            fhead_size=struct.calcsize ("l")
            buf, addr=UDPServerSocket.recvfrom (fhead_size)

            if buf:
                #returns tuple
                data_size=struct.unpack("l",buf)[0]

                while not recv_size ==data_size:
                    if data_size - recv_size>1024:
                        data,addr=UDPServerSocket.recvfrom(1024)
                        recv_size+=len(data)
                        print("\nTotal size of message received through buffer till this stream "+str(recv_size)+" (size 1024)")
                        count+=1

                    else:
                        data,addr=UDPServerSocket.recvfrom(data_size-recv_size)
                        recv_size=data_size
                        print("\nTotal size of message received through buffer till this stream "+str(recv_size)+" (size "+str(data_size-recv_size)+")")
                        count+=1

                    data_total+=data

    else:
        clicked()

tk.Label(window, text="Server Login", font="Courier 20 bold").place(relx=0.5, rely=0.05, anchor='center')

username = tk.Label(window, text="Username", font="Courier 15")
username.grid(row=70, column=10, padx=30, pady=100, sticky="e")
password = tk.Label(window, text="Password", font="Courier 15")
password.grid(row=72, column=10, padx=30, sticky="e")

username_box = tk.Entry(window, textvariable=username_value)
username_box.grid(row=70, column=15, sticky="w")
password_box = tk.Entry(window, textvariable=password_value, show="*")
password_box.grid(row=72, column=15, sticky="w")

tk.Button(text="Login", command=submit_login, font="Courier 15").place(relx=0.5, rely=0.5, anchor=tk.CENTER)

window.grid_columnconfigure(10, weight=1)
window.grid_columnconfigure(15, weight=1)
window.mainloop()
